# Advanced TFX

As we saw most of the simpler ML works can be done using generic pipeline components provided by the TFX. But in some cases we need to go beyond the generic component flows and should have complex compoents. Here we are focusing on techniques we can use in such scenarios.


### Training multiple models simultaneously

In many production systems, it if often required to train more than one model. And TFX provide support to train several models using a single pipeline. In such cases all the data preparation/ validation and transformation steps remains the same, but from there onwards data may flow into different types of models. We can do this by defining several trainer components in the TFX pipeline.

In [ ]:
def set_trainer(module_file, instance_name,
                train_steps=5000, eval_steps=100):
    return Trainer(
        module_file=module_file,
        custom_executor_spec=executor_spec.ExecutorClassSpec(
            GenericExecutor),
        examples=transform.outputs['transformed_examples'],
        transform_graph=transform.outputs['transform_graph'],
        schema=schema_gen.outputs['schema'],
        train_args=trainer_pb2.TrainArgs(num_steps=train_steps),
        eval_args=trainer_pb2.EvalArgs(num_steps=eval_steps),
        instance_name=instance_name)

Like the above rough code segment, we can define different trainer components with different module files. By doing this we can branch out and build complex ML pipelines than a single path pipeline.

<center><image src="imgs/6.jpg" width="500"/></center>

### TFLite model Exporting

Mobile application development is one of the most major task in today world software engineering tasks. When deploying ML models to such applications we have problems such as low computational power, low storage, restrained power usage etc. To use in such cases we can use TFLite models. We can convert our exising models as a part of our pipeline to TFLite models. But it should be noted that, not all TF operations are supported by the TFLite models. So should expect problems in converting problems if model uses complicated operations.

### Warm Start Model Training

